<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-dependencies" data-toc-modified-id="Load-dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load dependencies</a></span></li><li><span><a href="#Read-csv-files-to-DFs" data-toc-modified-id="Read-csv-files-to-DFs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read csv files to DFs</a></span></li><li><span><a href="#Fill-missing-values" data-toc-modified-id="Fill-missing-values-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fill missing values</a></span></li><li><span><a href="#Add-date-part" data-toc-modified-id="Add-date-part-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add date part</a></span></li><li><span><a href="#Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc." data-toc-modified-id="Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Add extra columns (purchased on weekend, monthend, month_diff etc.</a></span></li><li><span><a href="#One-hot-encoding-category_1-&amp;-authorized" data-toc-modified-id="One-hot-encoding-category_1-&amp;-authorized-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>One hot encoding category_1 &amp; authorized</a></span></li><li><span><a href="#Aggregate-by-card_id" data-toc-modified-id="Aggregate-by-card_id-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Aggregate by card_id</a></span></li><li><span><a href="#Aggregate-by-month-on-card_id-on-purchase_amount-(not-working)" data-toc-modified-id="Aggregate-by-month-on-card_id-on-purchase_amount-(not-working)-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Aggregate by month on card_id on purchase_amount (not working)</a></span></li><li><span><a href="#Successive-aggregates" data-toc-modified-id="Successive-aggregates-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Successive aggregates</a></span></li><li><span><a href="#Add-extra-cols-on-agg" data-toc-modified-id="Add-extra-cols-on-agg-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Add extra cols on agg</a></span></li><li><span><a href="#Load-test-&amp;-train-DFs" data-toc-modified-id="Load-test-&amp;-train-DFs-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Load test &amp; train DFs</a></span></li><li><span><a href="#Add-date-part-to-test-&amp;-train-dfs" data-toc-modified-id="Add-date-part-to-test-&amp;-train-dfs-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Add date part to test &amp; train dfs</a></span></li><li><span><a href="#Merge-train-&amp;-test-with-new-&amp;-old-transactions-history" data-toc-modified-id="Merge-train-&amp;-test-with-new-&amp;-old-transactions-history-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Merge train &amp; test with new &amp; old transactions history</a></span></li><li><span><a href="#Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc" data-toc-modified-id="Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Add extra columns like age, total transactions, installments, purchase amount, first buy etc</a></span></li><li><span><a href="#Mark-the-outliers" data-toc-modified-id="Mark-the-outliers-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Mark the outliers</a></span></li><li><span><a href="#Target-encode-the-outliers" data-toc-modified-id="Target-encode-the-outliers-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Target encode the outliers</a></span></li></ul></div>

1. Load dependencies
2. Read csv files to dataframes
3. Fill missing values
4. Add date part 
5. Add extra columns (purchased on weekend, monthend, month_diff etc.
6. Aggregate by card_id
7. Aggregate by categories
8. Mark categorical columns
9. Add exta interpreted columns on aggregates
10. Load test & train csvs to dfs
11. Add date part to test & train dfs
12. Merge train & test with new & old transactions history
13. Add extra columns like age, total transactions, installments, purchase amount, first buy etc
14. Mark the outliers
15. Target encode the outliers 
16. Save to feather

### Load dependencies

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather

### Read csv files to DFs

In [3]:
PATH = '../data/elo/'

In [4]:
files = ['historical_transactions', 'new_merchant_transactions']

In [5]:
hist_trans, new_hist_trans = [pd.read_csv(f'{PATH}{c}.csv') for c in files]

### Fill missing values

In [6]:
def fill_nas_for_transactions_df(df):
    # Fill nas for category_3 with mode
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    return df

In [7]:
dfs = [hist_trans, new_hist_trans]

In [8]:
hist_trans, new_hist_trans = [fill_nas_for_transactions_df(df) for df in dfs]

### Add date part

In [9]:
add_datepart(hist_trans, 'purchase_date', drop=False)
add_datepart(new_hist_trans, 'purchase_date', drop=False)

### Add extra columns (purchased on weekend, monthend, month_diff etc.

In [10]:
def add_extra_cols(df):
    df['purchased_on_weekend'] = (df.purchase_Dayofweek >=5).astype(int)
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    return df

In [11]:
hist_trans, new_hist_trans = [add_extra_cols(df) for df in dfs]

In [12]:
hist_trans.shape, new_hist_trans.shape

((29112361, 29), (1963031, 29))

### One hot encoding category_1 & authorized

In [13]:
def one_hot_encode(df):
    return pd.get_dummies(df, columns=['authorized_flag'])

In [14]:
# hist_trans, new_hist_trans = [one_hot_encode(df) for df in dfs]

In [15]:
hist_trans.to_feather('hist_trans')
new_hist_trans.to_feather('new_hist_trans')
hist_trans = feather.read_dataframe('hist_trans')
new_hist_trans = feather.read_dataframe('new_hist_trans')

### Aggregate by card_id

In [18]:
def aggregate_by_card_id(df):
    unique_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 
                   'subsector_id', 'purchase_Year', 'purchase_Month', 'purchase_Week', 'purchase_Day']
    aggs = {}
    for c in unique_cols:
        aggs[c] = ['nunique'] 
    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min', np.ptp]
    aggs['month_lag'] = ['max','min','mean','var']
    aggs['month_diff'] = ['mean', 'std', 'var']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['purchased_on_weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']
    cols_vals = df.columns.values
    for col in ['category_2','category_3', 'authorized_flag']:
        if col in cols_vals:
            df[col+'_amount_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
            df[col+'_amount_sum'] = df.groupby([col])['purchase_amount'].transform('sum')
            aggs[col+'_amount_mean'] = ['mean']
            aggs[col+'_amount_sum'] = ['sum']

    new_df = df.groupby(['card_id']).agg(aggs)
    new_df.columns = ['_'.join(col).strip() for col in new_df.columns.values]
    new_df.reset_index(inplace=True)
    other_df = (df.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    new_df = pd.merge(other_df, new_df, on='card_id', how='left')
    new_df["purchase_date_ptp"] = new_df["purchase_date_ptp"].dt.days
    new_df['purchase_date_diff'] = (new_df['purchase_date_max'] - new_df['purchase_date_min']).dt.days
    new_df['purchase_date_average'] = new_df['purchase_date_diff']/new_df['card_id_size']
    new_df['purchase_date_uptonow'] = (datetime.datetime.today() - new_df['purchase_date_max']).dt.days
    return new_df

In [19]:
import gc
gc.collect()

0

In [20]:
%time hist_trans_agg, new_hist_trans_agg = [aggregate_by_card_id(df) for df in [hist_trans, new_hist_trans]]

CPU times: user 3min 25s, sys: 22.1 s, total: 3min 48s
Wall time: 2min 37s


### Aggregate by month on card_id on purchase_amount (not working)

In [21]:
def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_diff'])

    agg_func = {
            'purchase_amount': ['count', 'sum', 'mean', 'min', 'max'],
            'installments': ['count', 'sum', 'mean', 'max'],
            }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean'])
    final_group.columns = ['_s'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group

In [22]:
[hist_trans_agg_s, new_hist_trans_agg_s] = [aggregate_per_month(df) for df in [hist_trans, 
                                                                             new_hist_trans]]

In [23]:
for df in [hist_trans_agg_s, new_hist_trans_agg_s]:
    df.drop('month_diff_smean', inplace=True, axis=1)

In [24]:
hist_trans_agg_s.head().T

,0,1,2,3,4
card_id,C_ID_00007093c1,C_ID_0001238066,C_ID_0001506ef0,C_ID_0001793786,C_ID_000183fdda
purchase_amount_count_smean,74.5,61.5,33,108,72
purchase_amount_sum_smean,-38.4225,-36.2236,-17.3009,-18.393,-34.419
purchase_amount_mean_smean,-0.486312,-0.592344,-0.607181,-0.183747,-0.494108
purchase_amount_min_smean,-0.725119,-0.731829,-0.739958,-0.734962,-0.731956
purchase_amount_max_smean,0.907511,0.190904,0.422155,4.09272,1.54944
installments_count_smean,74.5,61.5,33,108,72
installments_sum_smean,96,99,0.5,2.5,132
installments_mean_smean,1.30113,1.57587,0.00793651,0.0463563,1.66626
installments_max_smean,4.5,6.5,0.5,1,8


In [25]:
hist_trans_agg.shape, hist_trans_agg_s.shape

((325540, 47), (325540, 10))

### Successive aggregates

### Add extra cols on agg

In [26]:
def add_extra_cols_on_agg(df):
    df['inverse_avg_transactions_per_day'] = df['purchase_date_diff']/df['card_id_size']
    df['days_since_last_transaction'] = (datetime.datetime.today() - df['purchase_date_max']).dt.days
    df['repurchase_merchant_rate'] = df['transactions_count']/df['merchant_id_nunique']
    df['merchant_category_repurchase'] = df['merchant_category_id_nunique']/df['merchant_id_nunique']
    df['category_2_amount_sum_sum'] = df['category_2_amount_sum_sum']/df['transactions_count']
    df['category_3_amount_sum_sum'] = df['category_3_amount_sum_sum']/df['transactions_count']
    df['avg_spend_per_merchant'] = df['purchase_amount_sum']/df['merchant_id_nunique']
    df['avg_trans_per_merchant'] = df['transactions_count']/df['merchant_id_nunique']
    df['avg_spend_per_transaction'] = df['purchase_amount_sum']/df['transactions_count']
    return df

In [27]:
[hist_trans_agg, new_hist_trans_agg] = [add_extra_cols_on_agg(df) for df in [hist_trans_agg, 
                                                                             new_hist_trans_agg]]

In [28]:
hist_trans_agg.to_feather('hist_trans_agg_n')
new_hist_trans_agg.to_feather('new_hist_trans_agg_n')

In [29]:
hist_trans_agg = feather.read_dataframe('hist_trans_agg_n')
new_hist_trans_agg = feather.read_dataframe('new_hist_trans_agg_n')

### Load test & train DFs

In [30]:
PATH = 'data/elo/'

In [31]:
train, test = [pd.read_csv(f'{PATH}{c}') for c in ['train.csv', 'test.csv']]

### Add date part to test & train dfs

In [32]:
add_datepart(train, 'first_active_month', drop=False)
add_datepart(test, 'first_active_month', drop=False)

### Merge train & test with new & old transactions history

In [33]:
def join_dfs(left, right, left_on, right_on=None, suffix='_old'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("", suffix))

In [34]:
train_df = join_dfs(train, new_hist_trans_agg, left_on='card_id')

In [35]:
train_df = join_dfs(train_df, hist_trans_agg, left_on='card_id', suffix='_old')

In [36]:
train_df.shape

(201917, 125)

In [37]:
# train_df = join_dfs(train_df, hist_trans_agg_s, left_on='card_id', suffix='_old')

In [38]:
test_df = join_dfs(test, new_hist_trans_agg, left_on='card_id')
test_df = join_dfs(test_df, hist_trans_agg, left_on='card_id', suffix='_old')
# test_df = join_dfs(test_df, new_hist_trans_agg_s, left_on='card_id')

In [39]:
test_df.shape, train_df.shape

((123623, 124), (201917, 125))

### Add extra columns like age, total transactions, installments, purchase amount, first buy etc

In [40]:
for df in [train_df, test_df]:
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['card_id_total'] = df['card_id_size']+df['card_id_size_old']
    df['purchase_amount_total'] = df['purchase_amount_sum']+df['purchase_amount_sum_old']
    df['installments_total'] = df['installments_sum'] + df['installments_sum_old']
    df['hist_first_buy'] = (df['purchase_date_min_old'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['purchase_date_min'] - df['first_active_month']).dt.days
    df['avg_spend_per_transaction'] = df['purchase_amount_total']/df['card_id_total']

### Mark the outliers

In [41]:
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -31, 'outliers'] = 1
train_df['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

### Target encode the outliers 

In [42]:
train_df.shape, test_df.shape

((201917, 132), (123623, 130))

In [43]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = train_df.groupby([f])['outliers'].mean()
    train_df[f] = train_df[f].map(order_label)
    test_df[f] = test_df[f].map(order_label)

In [44]:
train_df.to_feather('train_df')
test_df.to_feather('test_df')

In [103]:
# len(np.unique(train_df.columns.values))

In [104]:
# u, c = np.unique(train_df.columns.values, return_counts=True)

In [105]:
# u[c > 1]

In [108]:
train_df.columns.values

array(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target',
       'first_active_monthYear', 'first_active_monthMonth', 'first_active_monthWeek',
       'first_active_monthDay', 'first_active_monthDayofweek', 'first_active_monthDayofyear',
       'first_active_monthIs_month_end', 'first_active_monthIs_month_start',
       'first_active_monthIs_quarter_end', 'first_active_monthIs_quarter_start',
       'first_active_monthIs_year_end', 'first_active_monthIs_year_start', 'first_active_monthElapsed',
       'transactions_count', 'city_id_nunique', 'merchant_category_id_nunique', 'merchant_id_nunique',
       'state_id_nunique', 'subsector_id_nunique', 'purchase_Year_nunique', 'purchase_Month_nunique',
       'purchase_Week_nunique', 'purchase_Day_nunique', 'purchase_amount_sum', 'purchase_amount_max',
       'purchase_amount_min', 'purchase_amount_mean', 'purchase_amount_var', 'installments_sum',
       'installments_max', 'installments_min', 'installments_mean

In [107]:
# train_df[['month_diff_mean', 'month_diff_std_old', 'month_diff_smean']]

In [109]:
train_df.head().T

,0,1,2,3,4
first_active_month,2017-06-01 00:00:00,2017-01-01 00:00:00,2016-08-01 00:00:00,2017-09-01 00:00:00,2017-11-01 00:00:00
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,0.0131446,0.0107116,0.0106099,0.0107116,0.00805849
feature_2,0.00875212,0.0113848,0.00875212,0.0141664,0.0141664
feature_3,0.0114275,0.0102828,0.0102828,0.0102828,0.0102828
target,-0.820283,0.392913,0.688056,0.142495,-0.159749
first_active_monthYear,2017,2017,2016,2017,2017
first_active_monthMonth,6,1,8,9,11
first_active_monthWeek,22,52,31,35,44
first_active_monthDay,1,1,1,1,1
